In [1]:
# PyAlex and Data Creation imports
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
import pyalex, os, csv
pyalex.config.email = "b.cliff@gwmail.gwu.edu"
from itertools import chain

In [2]:
# Data Validation imports
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, List

#### Configuring Output Files

In [3]:
WORKS_FILE = './data/works.csv'
AUTHORS_FILE = './data/authors.csv'
TOPICS_FILE = './data/topics.csv'
WORK_AUTH_FILE = './data/work_auth_edges.csv'

In [5]:
# Removing any old/vestige files
for path in [WORKS_FILE, AUTHORS_FILE, TOPICS_FILE, WORK_AUTH_FILE]:
    if os.path.exists(path):
        os.remove(path)

In [6]:
works_file = open(WORKS_FILE, 'w', newline='',encoding='utf-8')
authors_file = open(AUTHORS_FILE, 'w', newline='', encoding='utf-8')
topics_file = open(TOPICS_FILE, 'w', newline='', encoding='utf-8')
wa_edge_file = open(WORK_AUTH_FILE, 'w', newline='', encoding='utf-8')

In [7]:
# Creating the headers
works_writer = csv.writer(works_file)
authors_writer = csv.writer(authors_file)
topics_writer = csv.writer(topics_file)
wa_edges_writer = csv.writer(wa_edge_file)

In [ ]:
# Writing the headers to the csv files
works_writer.writerow(['id', 'doi', 'title', 'publication_date', 'type', 'related_works', 'corresponding_author_ids'])
authors_writer.writerow(['id', 'name', 'works_count', 'cited_by_count', 'affiliations', 'topics', 'counts_by_year'])
wa_edges_writer.writerow(['work_id', 'author_id', 'author_position', 'is_corresponding'])

30

#### Creating Pydantic Models for Validation

In [9]:
class WorksModel(BaseModel):
    id: str
    doi: Optional[str] = None
    title: str
    publication_date: str
    type: Optional[str] = None
    related_works: List[str] = None
    corresponding_author_ids: List[str] = None

In [10]:
class AuthorsModel(BaseModel):
    id: str
    display_name: str
    works_count: int
    cited_by_count: int
    affiliations: List[dict] = None
    topics: List[dict] = None
    counts_by_year: List[dict]

In [11]:
class TopicsModel(BaseModel):
    id: str
    display_name: str
    description: str
    keywords: List[str] = None
    subfield_name: Optional[str] = None
    field_name: Optional[str] = None
    domain_name: Optional[str] = None
    works_count: int
    cited_by_count: int
    updated_date: str
    created_date: str

### Data retrieval from OpenAlex

In [12]:
oa_replace = 'https://openalex.org/'

In [13]:
works_pager = Works().select(['id', 'doi', 'title',
'publication_date', 'type', 'related_works', 'corresponding_author_ids']).paginate(per_page=200)

In [14]:
authors_pager = Authors().select(['id', 'orcid', 'display_name',
                                  'works_count', 'cited_by_count',
                                  'affiliations', 'topics', 'counts_by_year']).paginate(per_page=200)

In [ ]:
topics_pager = Topics().select(['id', 'display_name', 'description', 'keywords',
                                'subfield', 'field', 'domain', 'works_count',
                                'cited_by_count', 'updated_date', 'created_date']).paginate(per_page=200)
# not including siblings for now - complicated

#### Data Cleaning

In [15]:
for page in works_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            raw['related_works'] = [rel.replace(oa_replace, '') for rel in raw['related_works']]
            raw['corresponding_author_ids'] = [rel.replace(oa_replace, '') for rel in raw['corresponding_author_ids']]
            w = WorksModel(**raw)
            works_writer.writerow(
                [w.id,
                w.doi,
                w.title,
                w.publication_date,
                w.type,
                w.related_works,
                w.corresponding_author_ids]
            )
        except (ValidationError, KeyError) as e:
            print("Skipping invalid record:", e)
            continue
works_file.close()


In [16]:
for page in authors_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            a = AuthorsModel(**raw)
            authors_writer.writerow([
                a.id,
                a.display_name,
                a.works_count,
                a.cited_by_count,
                a.affiliations,
                a.topics,
                a.counts_by_year
            ])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid record:", e)
            continue
authors_file.close()

In [17]:
topics_file.close()

In [ ]:
rand_A = Authors().random()

#### Including Filter and Select

In [ ]:
# Determining the keys for each Works() object
Works()['W2889162861'].keys()

In [ ]:
Works()['W2889162861']['abstract_inverted_index']

In [19]:
for work in Works().get():
    if work['authorships'] != []:
        print(work)
        break

{'id': 'https://openalex.org/W3038568908', 'doi': 'https://doi.org/10.1585/pfr.15.2402039', 'title': 'Radiation Resistant Camera System for Monitoring Deuterium Plasma Discharges in the Large Helical Device', 'display_name': 'Radiation Resistant Camera System for Monitoring Deuterium Plasma Discharges in the Large Helical Device', 'publication_year': 2020, 'publication_date': '2020-06-08', 'ids': {'openalex': 'https://openalex.org/W3038568908', 'doi': 'https://doi.org/10.1585/pfr.15.2402039', 'mag': '3038568908'}, 'language': 'en', 'primary_location': {'id': 'doi:10.1585/pfr.15.2402039', 'is_oa': True, 'landing_page_url': 'https://doi.org/10.1585/pfr.15.2402039', 'pdf_url': 'https://www.jstage.jst.go.jp/article/pfr/15/0/15_2402039/_pdf', 'source': {'id': 'https://openalex.org/S46033839', 'display_name': 'Plasma and Fusion Research', 'issn_l': '1880-6821', 'issn': ['1880-6821'], 'is_oa': True, 'is_in_doaj': False, 'is_core': True, 'host_organization': 'https://openalex.org/P4328135220',

#### Relationship Creation attempt

In [22]:
class WorkAuthoredByEdge(BaseModel):
    work_id: str
    author_id: str
    author_position: str
    is_corresponding: bool

In [20]:
works_pager = Works().select(['id', 'authorships']).paginate(per_page=200)

In [23]:
for page in works_pager:
    for raw in page:
        work_id = raw.get('id')
        if not work_id:
            continue
        
        work_id = work_id.replace(oa_replace, '')

        for auth in raw.get('authorships') or []:
            author_block = auth.get("author") or {}
            author_id = author_block.get("id")

            if not author_id:
                continue

            try:
                edge = WorkAuthoredByEdge(
                    work_id=work_id,
                    author_id=author_id.replace(oa_replace, ""),
                    author_position=auth.get("author_position"),
                    is_corresponding=auth.get("is_corresponding")
                )

                wa_edges_writer.writerow([
                    edge.work_id,
                    edge.author_id,
                    edge.author_position,
                    edge.is_corresponding
                ])

            except ValidationError:
                continue


wa_edge_file.close()